In [ ]:
using Revise

In [ ]:
push!(LOAD_PATH, "/home/amir/work/mps/src/")

using ExactDiagonalizationTools
using MatrixProductStateTools
using TensorNetAlgs
using LaTeXStrings
using Plots
using LsqFit

In [ ]:
lx = 61
tol = 1.e-9
maxdim = 16

# Find the ground sate Ising Transverse Field model
mps = MatrixProductState{Float64}(lx, 2, noise=0.2);
mpo = qitf_mpo(Float64, lx, 2, 0.9)
env = initialenv(mps, mpo)

for sweep = 1:4
    println("Starting sweep number $sweep")
    dmrg2sitesweep!(mps, mpo, env, maxdim, tol, verbose=false)
end
println("Done with DMRG!")

sz = Matrix(sz_half)
data = []
sx = Matrix(sp_half + sm_half)/2.
half = div(lx, 2)+1
apply_1siteoperator!(mps, half, 2 .*sx)
push!(data, measure_1point(mps, sz))

dt = 0.05
tfinal = 15.0

mpsc = convert(MatrixProductState{ComplexF64}, mps)
mpoc = convert(MatrixProductOperator{ComplexF64}, mpo)
env = initialenv(mpsc, mpoc)

for t=dt:dt:tfinal
    tdvp2sitesweep!(dt, mpsc, mpoc, env, maxdim=maxdim, tol=tol, verbose=false)
    if round(2*t) ≈ 2*t 
        println("time : $t")
        push!(data, real.(measure_1point(mpsc, sz)))
    end
end

In [ ]:
p = plot(title="TDVP evolution gs of QITF, L=$lx",
    xaxis = L"\ell", yaxis=L"$S^z(\ell)$", legend=false)
for i=1:length(data)
    plot!(p, 1:lx, -data[i])#, #label=latexstring("t=$((i-1)/2)"))
end
p

In [ ]:
function findsigmas(data)
    lx = length(data[1])
    nts = length(data)
    half = div(lx,2)+1
    @. fitfn(x, p) = p[1] * 1/(p[2]*sqrt(2*pi)) * exp(-1/2. * ((x-half)/p[2])^2)

    ps = zeros(2, nts)
    for n=1:nts
        fit = curve_fit(fitfn, collect(1:lx), data[n], [1., 1.])
        ps[:,n] = coef(fit)
    end
    ps[2,:]
end

In [ ]:
ts = 1:0.5:tfinal
plt=plot(xaxis=(L"t"), yaxis=(L"\sigma^2"), legend=:topleft)
sigmas = findsigmas(-data)
plot!(plt, ts, (sigmas.^2)[1:length(ts)], l=false, m=:c, label=latexstring("L=$lx"))